In [13]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle as plk
import importlib.util

###############################################################
# LINUX PATH

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/trainner/Documentos/Caio/GitReps/Photonics-training/Projects/Microring Resonator/LumericalFiles')) #Current directory

###############################################################
# WINDOWS PATH

#spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
#lumapi = importlib.util.module_from_spec(spec_win)
#os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
#sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/MMI/Lumericalfiles/'))
#spec_win.loader.exec_module(lumapi)

## Lumapi init

In [14]:
ring = lumapi.FDTD(filename='RINGProject.fsp',hide=False) 

## Functions

In [15]:
def RectInsert(X,XSPAN,Y,YSPAN,Z,ZSPAN,MATERIAL,lumerical):
    
    lumerical.switchtolayout()
    lumerical.addrect()
    lumerical.set({"x":X,"y":Y,"z":Z,
         "x span":XSPAN, "y span":YSPAN, "z span":ZSPAN,
         "material":MATERIAL
        })

## Assembly parameters

In [16]:
StRingLen = 1e-6 # Length of straight sections of the ring
Radius = 3e-6 # Ring radius

StraightSpan = 20e-6 # Length of straight waveguides
Gap = 150e-9 # Gap between waveguides and the ring

BaseHeight = 0.22e-6 # height of the waveguide
BaseWidth = 0.5e-6 # width of the waveguide base

## Assembly

In [33]:
ring.selectall()
ring.delete()
ring.addobject("ring_resonator")
ring.set({"x":-StraightSpan/2, "y":0, "z":0,
          "Lc":StRingLen,
          "x span":StraightSpan,
          "radius":Radius,
          "gap":Gap,
          "base height":BaseHeight,
          "base width":BaseWidth})

ring.setnamed("ring resonator::inner_bottom","enabled",0)
ring.setnamed("ring resonator::outer_bottom","enabled",0)
ring.setnamed("ring resonator::arc2","enabled",0)
ring.setnamed("ring resonator::arc3","enabled",0)

ring.addfdtd()
ring.set({"x":0, "y":Radius, "z":0,
          "x span":StraightSpan, "y span":2*Radius})
ring.addport()